### 层与块

### 多层感知机

In [5]:
import torch
from torch import nn
from torch.nn import functional as F

# 第一个线性层是20个输入，输出是256；最后的输出是10
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

# 2是批量的大小
X = torch.rand(2, 20) # X 是 二维的矩阵样本，每一个样本有20个特征 2*20
X, net(X)

(tensor([[0.4180, 0.3746, 0.7172, 0.8788, 0.5084, 0.7636, 0.9139, 0.8588, 0.0263,
          0.1307, 0.4978, 0.6148, 0.3253, 0.5089, 0.6024, 0.7119, 0.1554, 0.0993,
          0.3864, 0.6398],
         [0.8017, 0.8582, 0.7625, 0.6573, 0.4914, 0.0061, 0.9426, 0.9724, 0.0933,
          0.8904, 0.3935, 0.3477, 0.8834, 0.5204, 0.5007, 0.3283, 0.7755, 0.4699,
          0.0998, 0.4341]]),
 tensor([[ 3.5101e-01, -1.8300e-02, -5.9177e-02,  1.8375e-04, -1.6206e-01,
           9.2620e-02, -8.2704e-02, -2.0212e-01, -5.8795e-02, -4.0490e-01],
         [ 2.9098e-01, -4.5613e-02, -1.0497e-01, -4.4866e-02, -7.9504e-02,
           1.1406e-01, -9.6180e-02, -2.9171e-01, -7.7146e-03, -2.7772e-01]],
        grad_fn=<AddmmBackward0>))

#### Note: pytorch中module是一个很重要的概念

#### 任何一个层和神经网络都是module的一个子类

In [8]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏全连接层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义
        # 输入放入hidden层中，得到隐藏层的输出
        return self.out(F.relu(self.hidden(X)))

In [10]:
mlp_net = MLP()
mlp_net(X)

tensor([[ 0.0171, -0.2101,  0.2502,  0.0345, -0.0340, -0.2020,  0.0989, -0.0046,
          0.0090, -0.0487],
        [-0.0567, -0.3068,  0.2601, -0.1976, -0.0648, -0.1436, -0.0083,  0.1118,
         -0.0236, -0.1773]], grad_fn=<AddmmBackward0>)

### 顺序块

In [12]:
class MySequential(nn.Module):
    def __init__(self, *args): # args 是一个字典
        super().__init__()
        for block in args:
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[block] = block

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

# nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10) 放入args中，并按顺序放入modules这个成员变量中
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1570, -0.0307,  0.0909,  0.2949, -0.1601, -0.0307,  0.0768,  0.1117,
          0.1007,  0.0374],
        [-0.1799, -0.0961,  0.0768,  0.1976, -0.0690, -0.0544,  0.0738,  0.1530,
          0.0572,  0.0541]], grad_fn=<AddmmBackward0>)

### 正向传播函数中执行代码

In [14]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0300, grad_fn=<SumBackward0>)